## Import libraries

In [1]:
import os
import gspread
from google.oauth2.service_account import Credentials

import pandas as pd
from pandas import DataFrame
from typing import Optional, Literal, List

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Transform raw messages to insightful table

In [3]:
test_sample = [
    {
        "message": "Mẹ Bầu ăn có tốt không?",
        "inserted_at": "2024-09-02T07:43:27.000000",
        "from": "customer",
        "user": ["mẹ bầu", "trẻ con"],
        "purpose": ["tẩm bổ"],
    },
    {
        "message": "Đặt Súp Bào Ngư thăm người Ốm!",
        "inserted_at": "2024-08-26T11:53:24.000000",
        "from": "customer",
        "user": ["người bệnh", "mẹ bầu"],
        "purpose": ["thăm bệnh"],
    },
]

In [4]:
def create_dataframe(messages: List[dict]) -> DataFrame:
    """
    Creates a Pandas DataFrame from a list of messages.

    This function takes a list of dictionaries representing messages, each containing the following keys:
    - `message`: The message content.
    - `from`: The sender of the message.
    - `inserted_at`: The timestamp of the message insertion.
    - `user`: The user associated with the message.
    - `purpose`: The purpose of the message.

    The function converts the list of dictionaries into a Pandas DataFrame and performs the following transformations on the 'inserted_at' column:

    1. Converts the string representation of the timestamp to a datetime object.
    2. Localizes the timestamp to UTC.
    3. Converts the timestamp to the Asia/Bangkok timezone.

    Args:
        messages (List[dict]): A list of dictionaries representing messages.

    Returns:
        DataFrame: A Pandas DataFrame containing the messages.
    """
    df = pd.DataFrame(data=messages)

    # convert string to datetime
    df['inserted_at'] = df['inserted_at'].str.replace('T', ' ')
    df['inserted_at'] = df['inserted_at'].str.replace(r'\.\d{6}', '', regex=True)
    df['inserted_at'] = pd.to_datetime(df['inserted_at'], format='%Y-%m-%d %H:%M:%S')
    
    # transform timezone
    df['inserted_at'] = df['inserted_at'].dt.tz_localize('UTC')
    df['inserted_at'] = df['inserted_at'].dt.tz_convert('Asia/Bangkok')

    return df

In [5]:
df = create_dataframe(test_sample)

In [6]:
def quantify_data(message_df: DataFrame, user_col: str = 'user', purpose_col: str = 'purpose') -> DataFrame:
    user = message_df[[user_col]]
    count_user = user.explode([user_col])
    count_user = count_user.value_counts(user_col, ascending=False).reset_index()

    purpose = message_df[[purpose_col]]
    count_purpose = purpose.explode([purpose_col])
    count_purpose = count_purpose.value_counts(purpose_col, ascending=False).reset_index()

    return count_user, count_purpose

In [7]:
user, purpose = quantify_data(df)

display(user)

display(purpose)

,user,count
0,mẹ bầu,2
1,người bệnh,1
2,trẻ con,1


,purpose,count
0,thăm bệnh,1
1,tẩm bổ,1


## Load & Update Google sheet

In [3]:
scopes = [
    "https://www.googleapis.com/auth/spreadsheets"
]

creds = Credentials.from_service_account_file('../credentials.json', scopes=scopes)
client = gspread.authorize(creds)

In [4]:
sheet_id = os.environ['SHEET_ID']
sheet = client.open_by_key(sheet_id)

### Load worksheet

In [5]:
def load_worksheet(sheet_id: Optional[str] = None, sheet_idx: Optional[int] = None) -> DataFrame:
    if sheet_id is None and sheet_idx is None:
        raise Exception("Please provide either a sheet id or sheet index.")

    worksheet = None
    if sheet_id:
        worksheet = sheet.get_worksheet_by_id(sheet_id)
    else:
        worksheet = sheet.get_worksheet(sheet_idx)

    return pd.DataFrame(worksheet.get_all_records())

In [6]:
df = load_worksheet(sheet_idx=0)
df

,Col 1,Col 2
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


### Update worksheet

In [7]:
def update_worksheet(dataframe: DataFrame, sheet_id: Optional[str] = None, sheet_idx: Optional[int] = None, mode: Literal['update', 'replace'] = 'update') -> None:
    if sheet_id is None and sheet_idx is None:
        raise Exception("Please provide either `sheet_id` or `sheet_idx")
    
    # get worksheet
    if sheet_id:
        worksheet = sheet.get_worksheet_by_id(sheet_id)
    else:
        worksheet = sheet.get_worksheet(sheet_idx)

    # update worksheet
    if mode == 'replace':
        worksheet.clear()
    
    worksheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())

In [8]:
new_df = pd.DataFrame({'col a': [1,2,3,4,5], 'col b': [11,12,13,14,15]})
update_worksheet(new_df, sheet_idx=0, mode='replace')

In [9]:
df = load_worksheet(sheet_idx=0)
df

,col a,col b
0,1,11
1,2,12
2,3,13
3,4,14
4,5,15
